In [ ]:
import matplotlib.pyplot as plt
import scipy.signal
import numpy as np
import threading
import time
import usb
import sys

from sklearn.cluster import KMeans

import Jetson.GPIO as GPIO

from torch.nn import functional as F
from torch import nn
import torch

%matplotlib inline

In [ ]:
# our Autoencoder!
class AE(nn.Module):
    def __init__(self, ae_width):
        super(AE, self).__init__()

        self.fc1 = nn.Linear(1024, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, ae_width)
        self.fc4 = nn.Linear(ae_width, 64)
        self.fc5 = nn.Linear(64, 256)
        self.fc6 = nn.Linear(256, 1024)

    def encode(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

    def decode(self, x):
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        return self.fc6(x)

    def forward(self, x):
        x = self.encode(x.view(-1, 1024))
        return self.decode(x)

In [ ]:
# Load a trained model
model = AE(20)
model.load_state_dict(torch.load('Autoencoder_EEG.pt'))
model.eval()

# Push it onto the GPU
model.cuda()

In [ ]:
# Set up our NIA device
VENDOR_ID     = 0x1234   #: Vendor Id
PRODUCT_ID    = 0x0000   #: Product Id for the bridged usb cable
INTERFACE_ID  = 0x81     #: The interface we use to talk to the device
PACKET_LENGTH = 0x40     #: 64 bytes

class Nia():
    """ Attaches the NIA device, and provides low level data collection
    """ 

    def __init__(self, seconds=1) :
        self.read_length = 1024
        self.incoming_data = np.zeros(self.read_length*2, dtype=np.uint32)
        self.current_data = np.zeros(self.read_length, dtype=np.uint32)
        self.device = usb.core.find(idVendor=VENDOR_ID, idProduct=PRODUCT_ID)
        self.device.reset()
    
    def _bulk_read(self):
        """ Read data off the NIA from its internal buffer, of up to 16 samples"""
        read_bytes = self.device.read(INTERFACE_ID, PACKET_LENGTH, timeout=30)
        return read_bytes
        
    def _get_data(self):
        count = 0
        while True:
            bytes_data = self._bulk_read()
            point_count = int(bytes_data[54])

            for i in range(point_count):
                self.incoming_data[count + i] = int.from_bytes(bytes_data[i*3:i*3+3], byteorder='little')
               
            count = count + point_count

            if count >= self.read_length:
                break

        self.current_data = self.incoming_data[:self.read_length]
        
    def update(self):
        data_thread = threading.Thread(target=self._get_data)
        data_thread.start()
        data_thread.join()
        
def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        sos = scipy.signal.butter(order, [low, high], analog=False, btype='band', output='sos')
        return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        sos = butter_bandpass(lowcut, highcut, fs, order=order)
        y = scipy.signal.sosfilt(sos, data)
        return y

In [ ]:
# Capture data, clean it up and feed it through the pre-trained Autoencoder
def process_eeg_data():
    # Capture 250 ms of EEG Data, and run it through the Aurtoencoder

    nia.update()
    dataset_unfiltered = nia.current_data
    mean, std = np.mean(dataset_unfiltered), np.std(dataset_unfiltered)
    dataset_unfiltered = np.where(dataset_unfiltered > mean + std*10, dataset_unfiltered - 2**16, dataset_unfiltered)
    dataset_unfiltered = np.where(dataset_unfiltered < mean - std*10, dataset_unfiltered + 2**16, dataset_unfiltered)
    dataset_unfiltered = dataset_unfiltered - mean
    dataset_unfiltered = dataset_unfiltered / std

    # make sure we are the right data type
    dataset_unfiltered = dataset_unfiltered.astype(np.float32)
    dataset_filtered = butter_bandpass_filter(dataset_unfiltered, lowcut=5, highcut=35, fs=4096, order=6)
    dataset_filtered = dataset_filtered.astype(np.float32)
    
    with torch.no_grad():
        autoencoder = model.encode(torch.from_numpy(dataset_filtered).cuda())
    
    return autoencoder

In [ ]:
# instantiate our NIA class
if 'nia' in locals():
    nia.device.reset()
nia = Nia()


In [ ]:
# Record 25 seconds of data in State 1
state_1 = np.zeros((100, 20))
for i in range(100):
    data = process_eeg_data()
    state_1[i,:] = data.cpu().numpy()

In [ ]:
# Record 25 seconds of data in State 2
state_2 = np.zeros((100, 20))
for i in range(100):
    data = process_eeg_data()
    state_2[i,:] = data.cpu().numpy()

In [ ]:
# Combine the EEG data, and use it to train the Classifier
combined = np.vstack((state_1, state_2))
kmeans = KMeans(n_clusters=2, random_state=0).fit(combined)

In [ ]:
# Set up the GPIO pins on the Jetson nano
GPIO.setmode(GPIO.BOARD)
GPIO.setup([11, 12], GPIO.OUT)

In [ ]:
# Finally, run the Classifier, and control those LEDs!!!
while True:
    autoencoder_output = process_eeg_data().cpu().numpy()
    state = kmeans.predict(autoencoder_output.reshape(1, -1))

    if state == 0:
        GPIO.output(11, GPIO.HIGH)
        GPIO.output(12, GPIO.LOW)
    else:
        GPIO.output(12, GPIO.HIGH)
        GPIO.output(11, GPIO.LOW)